In [237]:
import pandas as pd
import datetime

In [238]:
caminho_do_arquivo = "https://raw.githubusercontent.com/Russel-Servicos/automacao-mkt-chat/main/planilha_chat_base.csv"
caminho_do_arquivo = "/content/planilha_chat_estagio_1de2_2024_09_18_16_25.csv"

df = pd.read_csv(caminho_do_arquivo)

data = "11/09/2024"

In [239]:
## Remove linhas duplicadas
df = df.drop_duplicates(subset = ["Name"])

## Remove colunas
df = df.drop(columns = ['Visitor ID', 'Initiated Time', 'IP', 'Mode'], axis = 1)

In [240]:
## Muda os tipos de dados das colunas
columns_to_type_conversions={
    'Page': 'str',
    'Question':'str',
    'Source': 'str',
    'Campaign Name': 'str',
  }
df = df.astype(columns_to_type_conversions)
df

,Name,Page,Question,Keyword,Campaign Medium,Campaign Name,Campaign Source,State,Source
0,Visitor 289024,https://russelservico.com.br/,Quero Contratar,NaN,NaN,nan,NaN,sao paulo,Referrals
1,Marilda,nan,Olá! Tudo bem? Gostaria de solicitar um orçame...,NaN,NaN,nan,NaN,NaN,WhatsApp
2,Visitor 578049,https://russelservico.com.br/terceirizacao/con...,Busco Emprego,NaN,NaN,nan,NaN,parana,Search Engine
3,Visitor 117196,https://russelservico.com.br/?utm_source=googl...,Busco Emprego,diarista,pesquisa,pacotelimpeza,google,sao paulo,Adwords
5,Visitor 717932,https://russelvagas.com.br/,Quero emprego.,NaN,NaN,nan,NaN,sao paulo,Direct
...,...,...,...,...,...,...,...,...,...
98,Visitor 209071,https://russelservico.com.br/?utm_source=googl...,Quero Contratar,empresa terceirizada,pesquisa,terceirizacao,google,alagoas,Adwords
99,Visitor 188139,https://russelservico.com.br/?utm_source=googl...,Busco Emprego,contratar operador empilhadeira,pesquisa,opmaquinas,google,mato grosso,Adwords
100,Visitor 215253,https://russelvagas.com.br/,Quero emprego.,NaN,NaN,nan,NaN,mato grosso,Direct
103,Visitor 495927,https://russelvagas.com.br/,FALE CONOSCO,NaN,NaN,nan,NaN,minas gerais,Direct


In [241]:
## Filtra as linhas a coluna Page (Página de Aterrissagem)
query_is_russelvagas = "not (Page.str.contains('russelvagas.com'))"
df = df.query(query_is_russelvagas)

In [242]:
## Renomeia colunas
columns = {
    "Name": "ID do Visitante",
    "Page": "Página de Aterrissagem",
    "Campaign Medium": "Meio",
    "Campaign Name": "Campanha",
    "Campaign Source": "Origem",
    "State": "Estado",
    "Source": "Canal"
}
df = df.rename(columns=columns)

In [243]:
# Cria novas colunas
df['Data'] = data
df['Qualificação'] = ''
df['Contagem'] = ''
df['Taxa de Conversão'] = ''

In [244]:
## Reorganiza colunas
df = df.reindex(['Data','ID do Visitante','Canal','Página de Aterrissagem','Question','Origem','Meio','Campanha','Estado','Keyword','Qualificação','Contagem','Taxa de Conversão'], axis = 1)
df.head()

,Data,ID do Visitante,Canal,Página de Aterrissagem,Question,Origem,Meio,Campanha,Estado,Keyword,Qualificação,Contagem,Taxa de Conversão
0,11/09/2024,Visitor 289024,Referrals,https://russelservico.com.br/,Quero Contratar,NaN,NaN,nan,sao paulo,NaN,,,
1,11/09/2024,Marilda,WhatsApp,nan,Olá! Tudo bem? Gostaria de solicitar um orçame...,NaN,NaN,nan,NaN,NaN,,,
2,11/09/2024,Visitor 578049,Search Engine,https://russelservico.com.br/terceirizacao/con...,Busco Emprego,NaN,NaN,nan,parana,NaN,,,
3,11/09/2024,Visitor 117196,Adwords,https://russelservico.com.br/?utm_source=googl...,Busco Emprego,google,pesquisa,pacotelimpeza,sao paulo,diarista,,,
7,11/09/2024,Visitor 707375,Adwords,https://russelservico.com.br/?utm_source=googl...,Busco Emprego,google,pesquisa,varejo,rio grande do norte,contratar vendedor,,,


In [245]:
## Atribui as linhas, o valor "Emprego"
query_is_emprego = "(Question.str.contains('Emprego', case=False))"
df_is_emprego = df.query(query_is_emprego)
df_is_emprego.loc[:, "Qualificação"] = "Emprego"

query_is_not_emprego = f"not {query_is_emprego}"
df_not_is_emprego = df.query(query_is_not_emprego)

df = pd.concat([df_is_emprego, df_not_is_emprego])

In [246]:
## Deletar linhas da coluna 'Página de Aterrissagem' que conter 'russelvagas'
df = df.drop(columns = ['Question'], axis = 1)

In [247]:
# Exportar como CSV
now = datetime.datetime.now()
now = now.strftime("%Y_%m_%d_%H_%M")
df.to_csv(f'./planilha_chat_estagio_1de2_{now}.csv', index=False)